# Lab #2

In [ ]:
from IPython.display import clear_output
from matplotlib import pyplot as plt
from PIL import Image
import cv2

## Camera

Let's start with configuring our camera.    
Initialize the camera with 300x300 pixels.

In [ ]:
def capture_photo():
    cam = cv2.VideoCapture(0, cv2.CAP_V4L2)

    for i in range(50):
        ret, img = cam.read()
    
    cam.release()

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.axis('off')
    plt.imshow(img)
    plt.savefig('data/capture.jpg')
    plt.show()
    return img


Now, let's capture an image with the camera and save it.   

In [ ]:
capture_photo()

image = Image.open("data/capture.jpg")
plt.imshow(image)

## Exploration

In this example we'll show how you can classify objects with Raspberry Pi5!   
We'll use a pre-trained neural network (Resnet18) that was trained on the ImageNet dataset to detect different common objects.   

These include:

- Mailbox (index 637)
- Cup (index 968)
- ...

First, we'll want to import the resnet18 class which takes the pre-trained weights as input.

In [ ]:
from torchvision.models import resnet18
from torchvision import transforms as T
import torch
import json

model = resnet18(pretrained=True). eval()

preprocess = T.Compose([
    T.Resize(256), 
    T.CenterCrop(224), 
    T.ToTensor(), 
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

classes = json.load(open('data/imagenet_labels.json'))

Then let's try with sample images:

In [ ]:
def classify_image(img):
    # preprocess the image
    batch = preprocess(img).unsqueeze(0)

    # perform inference 
    with torch.no_grad():
        pred = model(batch)
        prob = pred.softmax(dim=1)

    class_id = prob.argmax(dim=1).item()
    score = prob[0, class_id].item()

    # print the result
    print(f"{classes[class_id]}: {100 * score:.1f}%")
    return img

In [ ]:
# load the image 
img = Image.open('data/mailbox.jpg')

classify_image(img)

In [ ]:
# load the image 
img = Image.open('data/cup.png')

classify_image(img)

Now, let's run the model using our image.

In [ ]:
# load the image 
img = Image.open('data/capture.jpg')

classify_image(img)